In [1]:
#pip install selenium

In [2]:
#pip install webdriver_manager

In [3]:
#pip install html5lib

In [4]:
#pip install lxml

In [5]:
# Import statements
from bs4 import BeautifulSoup
import requests
import xlrd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import unittest
import time


In [6]:

class SubmitCoronaCases(unittest.TestCase):
    def setUp(self):
        #self.driver = webdriver.Chrome(r'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe')
        self.driver = webdriver.Chrome(ChromeDriverManager().install())

        
        
    def test_submitData(self):
        
        driver = self.driver
        driver.maximize_window()
        for row in range(1, len(data)):
            
            driver.get(destinationURL)
            self.assertEqual("Corona Update", driver.title)

            for col in range(0,4):
                print("Values are: " + str(data[0][col]) + " and " + str(data[row][col]))
                elem = driver.find_element_by_id(data[0][col])
                elem.clear
                elem.send_keys(data[row][col])
                time.sleep(1)

            elem = driver.find_element_by_id('continue')
            elem.send_keys(Keys.RETURN)
            time.sleep(10)
            obj = driver.switch_to.alert
            obj.accept()


    def tearDown(self):
        self.driver.quit()
        


In [7]:
# Initialize
URL="https://www.worldometers.info/coronavirus/country/"

destinationURL =r'http://localhost:8080/corona/index.html'
data=[]
data.append([])
data[0].append("country")
data[0].append("Total Cases")
#data[0].append("Active Cases")
#data[0].append("Closed Cases")



In [8]:

workbook = xlrd.open_workbook('InputCountry.xls')
worksheet = workbook.sheet_by_name('Input')
countries = []
col_idx = 0
for row_idx in range(0, worksheet.nrows):
        cell_obj = worksheet.cell(row_idx, col_idx)
        print ('(row %s) %s' % (row_idx, cell_obj.value))
        countries.append(cell_obj.value)


(row 0) canada
(row 1) india
(row 2) Spain


In [9]:

for country in countries: 
    rowData=[]
    print(country) 
    rowData.append(country)
    sourceURL = URL+country
    print(sourceURL)
    r=requests.get(sourceURL)
    # Read from above URL specified
    soup=BeautifulSoup(r.content, 'html.parser')

    totalCases = 0
    activeCases = 0
    closedCase = 0
    for main_div in soup.findAll('div',{'id':'maincounter-wrap'}):
        print(main_div)
        heading = main_div.find('h1')
        print (heading)
        #print (heading.contents[0])
        #print(type(heading))
        if(heading.contents[0] == 'Coronavirus Cases:'):
            print("This is Total cases")
            casesData = main_div.find('div', {'class':'maincounter-number'})
            print(casesData)
            caseValue = int(casesData.contents[1].contents[0].replace(',', ''))
            print(caseValue)
            totalCases = caseValue
            rowData.append(totalCases)

            break
    print("----------------------")

    for main_div in soup.findAll('div',{'class':'panel panel-default'}):
        #print(main_div)
        print("======")

        heading = main_div.find('div',{'class':'panel-heading'})
        print (heading)
        print (heading.contents[1].contents[0])
        print("xxxxxx")
        
        #print (heading.contents[0])
        #print(type(heading))
        if(heading.contents[1].contents[0] == '\nClosed Cases'):
            print("This is Closed cases")
            casesData = main_div.find('div',{'class':'panel-body'}).find('div',{'class':'panel_front'}).contents[1].contents[0]
            print(casesData)
            '''closedCases=int(casesData.replace(',', ''))
            print(closedCases)
            activeCases = totalCases - closedCases
            print(activeCases)

            rowData.append(activeCases)
            rowData.append(closedCases)
            '''

            break
            
        print("----------------------")
    data.append(rowData) 
    # Store data in the html 
    destinationURL ="http://localhost:8080/corona/index.html"

    


canada
https://www.worldometers.info/coronavirus/country/canada
<div id="maincounter-wrap" style="margin-top:15px">
<h1>Coronavirus Cases:</h1>
<div class="maincounter-number">
<span style="color:#aaa">4,669,364        </span>
</div>
</div>
<h1>Coronavirus Cases:</h1>
This is Total cases
<div class="maincounter-number">
<span style="color:#aaa">4,669,364        </span>
</div>
4669364
----------------------
<div class="panel-heading" style="text-align:center">
<span class="panel-title" style="font-size:18px; text-transform:uppercase; font-weight:100">
                            Active Cases
                        </span>
</div>

                            Active Cases
                        
xxxxxx
----------------------
<div class="panel-heading" style="text-align:center;">
<span class="panel-title" style="font-size:18px; text-transform:uppercase; font-weight:100">
                                    Closed Cases</span>
</div>

                                    Closed Cases
xxxxx

In [10]:
print(data)
#unittest.main()

[['country', 'Total Cases'], ['canada', 4669364], ['india', 44986461], ['Spain', 13845825]]
